In [1]:
import pandas as pd
from statsmodels.multivariate.manova import MANOVA
from scipy import stats

In [2]:
#TEST MANOVA PARA EVALUAR SI LA HIPOTESIS NULA DE LOS GRTUPOS NO TIENEN DIFERENCIAS MULTIVARIANTES.
#Un valor cercano a 1 indica que los grupos son similares; cercano a 0, los grupos son diferentes.
'''El Intercept representa el término constante en el modelo. Es el valor promedio de las variables 
dependientes cuando todas las variables independientes son cero (en este caso, antes de considerar la influencia de target)'''
def manova_test(df, name_columns, name):
    formula = ' + '.join(name_columns) + ' ~ target'
    manova = MANOVA.from_formula(formula, data=df)
    result = manova.mv_test()
    #Solo toma los resultados del target
    stat_table = result.results['target']['stat']
    df_stats = pd.DataFrame(stat_table).reset_index().rename(columns={'index': 'Test'})
    return df_stats

In [3]:
df=pd.read_csv('../../data/described/described_antiviral_homology_90.csv')

In [4]:
df['target'] = df['target'].astype(str)
df = df.rename(columns=lambda x: x.replace(" ", "_"))

In [5]:
df_high = df[df['target'] == '2']
df_mid = df[df['target'] == '1']
df_low = df[df['target'] == '0']

In [6]:
all_col = [
    'Molecular_Weight', 'Isoelectric_point', 'Charge_density', 'Charge',
    'Instability_index', 'Aromaticity', 'Aliphatic_index', 'Boman_index', 'Hydrophobic_ratio',
    'freq_A', 'freq_C', 'freq_D', 'freq_E', 'freq_F', 'freq_G', 'freq_H', 'freq_I',
    'freq_N', 'freq_K', 'freq_L', 'freq_M', 'freq_P', 'freq_Q', 'freq_R',
    'freq_S', 'freq_T', 'freq_V', 'freq_W', 'freq_Y'
]

In [7]:
property_columns = [
    'Molecular_Weight', 'Isoelectric_point', 'Charge_density', 'Charge',
    'Instability_index', 'Aromaticity', 'Aliphatic_index', 'Boman_index', 'Hydrophobic_ratio'
]

aminoacid_columns = [
    'freq_A', 'freq_C', 'freq_D', 'freq_E', 'freq_F', 'freq_G', 'freq_H', 'freq_I',
    'freq_N', 'freq_K', 'freq_L', 'freq_M', 'freq_P', 'freq_Q', 'freq_R',
    'freq_S', 'freq_T', 'freq_V', 'freq_W', 'freq_Y'
]

In [8]:
n_iterations = 1000
n_samples = min(len(df_high), len(df_mid), len(df_low)) 
i=0
results_list = []
post_hoc_results_list = []

In [9]:
res_prop=manova_test(df, property_columns, 'prop')
res_amino=manova_test(df, aminoacid_columns, 'amino')
res_all=manova_test(df, all_col, 'all')

In [10]:
res_prop_list= []
res_amino_list= []
res_all_list= []

In [11]:
for i in range(n_iterations):
    high_sample = df_high.sample(n=n_samples, replace=False, random_state=i)
    mid_sample = df_mid.sample(n=n_samples, replace=False, random_state=i)
    low_sample = df_low.sample(n=n_samples, replace=False, random_state=i)
    df_sample = pd.concat([high_sample, mid_sample, low_sample], axis=0)

    res_prop_list.append(manova_test(df_sample, property_columns, 'prop'))
    res_amino_list.append(manova_test(df_sample, aminoacid_columns, 'amino'))
    res_all_list.append(manova_test(df_sample, all_col, 'all'))

In [12]:
res_prop_list = pd.concat(res_prop_list, axis=0).reset_index(drop=True)
res_amino_list = pd.concat(res_amino_list, axis=0).reset_index(drop=True)
res_all_list = pd.concat(res_all_list, axis=0).reset_index(drop=True)

In [13]:
res_prop_list

,Test,Value,Num DF,Den DF,F Value,Pr > F
0,Wilks' lambda,0.867909,18,1166.0,4.754904,0.0
1,Pillai's trace,0.135308,18.0,1168.0,4.708545,0.0
2,Hotelling-Lawley trace,0.148488,18,967.895881,4.802795,0.0
3,Roy's greatest root,0.116738,9,584,7.574993,0.0
4,Wilks' lambda,0.867416,18,1166.0,4.774664,0.0
...,...,...,...,...,...,...
3995,Roy's greatest root,0.152446,9,584,9.892066,0.0
3996,Wilks' lambda,0.853073,18,1166.0,5.356924,0.0
3997,Pillai's trace,0.150218,18.0,1168.0,5.269522,0.0
3998,Hotelling-Lawley trace,0.168375,18,967.895881,5.446027,0.0


In [14]:
res_prop_list.to_csv("../../rest/manova_prop.csv", index=False)
res_amino_list.to_csv("../../rest/manova_amino.csv", index=False)
res_all_list.to_csv("../../rest/manova_all.csv", index=False)

In [15]:
cols = ["Value", "Num DF", "Den DF", "F Value", "Pr > F"]
res_prop_list[cols] = res_prop_list[cols].apply(pd.to_numeric, errors='coerce')
res_amino_list[cols] = res_amino_list[cols].apply(pd.to_numeric, errors='coerce')
res_all_list[cols] = res_all_list[cols].apply(pd.to_numeric, errors='coerce')

In [31]:
promedios = res_all_list.groupby('Test').mean(numeric_only=True).reset_index()
promedios.round(5)

,Test,Value,Num DF,Den DF,F Value,Pr > F
0,Hotelling-Lawley trace,0.40598,48.0,1046.87764,4.79634,0.0
1,Pillai's trace,0.32343,48.0,1138.00000,4.57904,0.0
2,Roy's greatest root,0.31063,24.0,569.00000,7.36460,0.0
3,Wilks' lambda,0.69712,48.0,1136.00000,4.68728,0.0


In [32]:
promedios_filtered = promedios[["Test", "Pr > F"]]
promedios_filtered

,Test,Pr > F
0,Hotelling-Lawley trace,1.933488e-15
1,Pillai's trace,1.118840e-14
2,Roy's greatest root,1.357728e-16
3,Wilks' lambda,4.189726e-15
